In [1]:
import torch
import torchvision

print("="*50)
print("PyTorch CUDA Installation Test")
print("="*50)

print(f"PyTorch version: {torch.__version__}")
print(f"Torchvision version: {torchvision.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")
    print(f"cuDNN version: {torch.backends.cudnn.version()}")
    print(f"Number of GPUs: {torch.cuda.device_count()}")

    for i in range(torch.cuda.device_count()):
        print(f"GPU {i}: {torch.cuda.get_device_name(i)}")

    print(f"Current GPU: {torch.cuda.current_device()}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

    # Test basic CUDA operations
    try:
        x = torch.randn(3, 3).cuda()
        y = torch.randn(3, 3).cuda()
        z = x @ y
        print("✅ Basic CUDA operations working!")
        print(f"Test tensor device: {z.device}")
    except Exception as e:
        print(f"❌ CUDA test failed: {e}")
else:
    print("❌ CUDA not available - CPU-only PyTorch installation detected")

print("="*50)

PyTorch CUDA Installation Test
PyTorch version: 2.8.0+cu128
Torchvision version: 0.23.0+cu128
CUDA available: True
CUDA version: 12.8
cuDNN version: 91002
Number of GPUs: 1
GPU 0: NVIDIA GeForce RTX 3070 Laptop GPU
Current GPU: 0
GPU Memory: 8.6 GB
✅ Basic CUDA operations working!
Test tensor device: cuda:0


In [6]:
#!/usr/bin/env python3
"""
Simple IKEA Product Search Program using ikea_api

Note: The ikea_api package is archived and no longer maintained.
Install with: pip install ikea_api[requests]
"""

import ikea_api
from ikea_api.wrappers import fetch, get_items
import json


class IkeaSearcher:
    def __init__(self, country="us", language="en"):
        """Initialize the IKEA searcher with country and language settings."""
        self.constants = ikea_api.Constants(country=country, language=language)
        self.search_api = ikea_api.Search(self.constants)

    def search_products(self, query, limit=10, types=None):
        """
        Search for products by name.

        Args:
            query (str): Search term (e.g., "Billy", "sofa", "lamp")
            limit (int): Maximum number of results (default: 10, max: 24)
            types (list): Search result types. Options:
                         ["PRODUCT", "CONTENT", "PLANNER", "REFINED_SEARCHES", "ANSWER"]

        Returns:
            dict: Search results from IKEA API
        """
        # Create search endpoint
        if types:
            endpoint = self.search_api.search(query, limit=limit, types=types)
        else:
            endpoint = self.search_api.search(query, limit=limit)

        # Execute the search using the requests runner
        runner = ikea_api.run_requests()
        results = runner.run(endpoint)

        return results

    def get_product_details(self, item_codes):
        """
        Get detailed information about specific products.

        Args:
            item_codes (list): List of IKEA item codes (e.g., ["30457903"])

        Returns:
            dict: Product details
        """
        ingka_items = ikea_api.IngkaItems(self.constants)
        endpoint = ingka_items.get_items(item_codes)

        runner = ikea_api.run_requests()
        return runner.run(endpoint)

    def get_stock_info(self, item_code):
        """
        Check stock availability for a product.

        Args:
            item_code (str): IKEA item code

        Returns:
            dict: Stock information
        """
        stock = ikea_api.Stock(self.constants)
        endpoint = stock.get_stock(item_code)

        runner = ikea_api.run_requests()
        return runner.run(endpoint)

    def format_search_results(self, results):
        """Format and display search results in a readable way."""
        if not results:
            print("   No results found.")
            return

        # Handle different response structures
        products = []

        # Try different possible result structures
        if isinstance(results, dict):
            if 'productListPage' in results:
                products = results.get('productListPage', {}).get('productWindow', [])
            elif 'products' in results:
                products = results.get('products', [])
            elif 'searchResultPage' in results:
                search_page = results.get('searchResultPage', {})
                products = search_page.get('products', []) or search_page.get('productWindow', [])

        if not products:
            print("   No products found in search results.")
            print(f"   Raw result keys: {list(results.keys()) if isinstance(results, dict) else 'Not a dict'}")
            return

        print(f"   Found {len(products)} products:")
        print("   " + "-" * 58)

        for i, product in enumerate(products, 1):
            try:
                name = product.get('name', product.get('displayName', 'Unknown Product'))

                # Handle different price structures
                price_info = product.get('price', {})
                if isinstance(price_info, dict):
                    current_price = price_info.get('currentPrice', price_info.get('price', 'N/A'))
                else:
                    current_price = price_info or 'N/A'

                item_code = product.get('itemCode', product.get('itemNo', product.get('id', 'N/A')))

                print(f"   {i}. {name}")
                print(f"      Price: {current_price}")
                print(f"      Item Code: {item_code}")

                # Get URL if available
                url = product.get('url', product.get('link'))
                if url and not url.startswith('http'):
                    url = f"https://www.ikea.com{url}"
                if url:
                    print(f"      URL: {url}")

                print()

            except Exception as e:
                print(f"   Error formatting product {i}: {e}")
                print(f"   Raw product data: {product}")
                print()


def main():
    """Main function to demonstrate IKEA product search."""
    print("IKEA Product Search Demo")
    print("=" * 30)

    # Initialize searcher (you can change country/language)
    searcher = IkeaSearcher(country="us", language="en")

    # Example searches
    search_terms = ["Billy", "sofa", "lamp"]

    for term in search_terms:
        print(f"\n🔍 Searching for: '{term}'")
        try:
            # Search for products
            results = searcher.search_products(term, limit=5)
            searcher.format_search_results(results)

        except Exception as e:
            print(f"Error searching for '{term}': {e}")

    # Example: Get details for a specific product
    print("\n📋 Getting details for a specific product...")
    try:
        # Billy bookcase item code example
        item_code = "00263850"  # This might not be current
        details = searcher.get_product_details([item_code])
        print(f"Product details for {item_code}:")
        print(json.dumps(details, indent=2)[:500] + "..." if len(str(details)) > 500 else json.dumps(details, indent=2))

    except Exception as e:
        print(f"Error getting product details: {e}")


def interactive_search():
    """Interactive search function."""
    searcher = IkeaSearcher(country="us", language="en")

    print("\nInteractive IKEA Search")
    print("Enter 'quit' to exit\n")

    while True:
        query = input("Enter search term: ").strip()

        if query.lower() in ['quit', 'exit', 'q']:
            print("Goodbye!")
            break

        if not query:
            continue

        try:
            results = searcher.search_products(query, limit=5)
            searcher.format_search_results(results)

        except Exception as e:
            print(f"Search error: {e}")


if __name__ == "__main__":
    # Run the demo
    main()

    # Uncomment the line below for interactive mode
    # interactive_search()

ImportError: cannot import name 'fetch' from 'ikea_api.wrappers' (C:\Users\lu-localadmin\PycharmProjects\Vqvaetrain\.venv\Lib\site-packages\ikea_api\wrappers\__init__.py)

In [4]:
import sys
!{sys.executable} -m pip install ikea_api

  Using cached ikea_api-2.4.0-py3-none-any.whl.metadata (9.7 kB)
Using cached ikea_api-2.4.0-py3-none-any.whl (30 kB)



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
try:
    import ikea_api
    print("✅ ikea_api imported successfully!")
except ImportError as e:
    print(f"❌ Still having issues: {e}")

✅ ikea_api imported successfully!


In [9]:
import requests
import json

# --- Configuration ---
# Define country and language codes for the API requests.
COUNTRY_CODE = "gb"  # Great Britain
LANGUAGE_CODE = "en" # English

# A dictionary of some UK store IDs.
# You can find more by inspecting network traffic on IKEA's website.
STORES = {
    "1": {"id": "149", "name": "Wembley"},
    "2": {"id": "376", "name": "Greenwich"},
    "3": {"id": "147", "name": "Croydon"},
    "4": {"id": "209", "name": "Manchester"},
    "5": {"id": "268", "name": "Edinburgh"},
}

# --- Main Script Logic ---

def get_guest_token():
    """
    Requests a guest authentication token from the IKEA API.
    This token is required for all other API calls.
    """
    token_url = "https://api.ingka.ikea.com/guest/token"
    headers = {
        "Content-Type": "application/json",
    }
    payload = {"retailUnit": COUNTRY_CODE}

    print("Requesting guest access token...")
    try:
        response = requests.post(token_url, headers=headers, json=payload)
        response.raise_for_status()
        data = response.json()
        if "access_token" in data:
            print("✅ Access token received.")
            return data["access_token"]
        else:
            print("🚨 'access_token' not found in response.")
            return None
    except requests.exceptions.RequestException as e:
        print(f"🚨 An error occurred while getting the token: {e}")
        return None

def search_product(query, token):
    """
    Searches for a product using the unofficial IKEA Search API.
    Returns the first product found or None if no products are found.
    """
    search_url = (
        f"https://api.ingka.ikea.com/cia/availabilities/search/v1?"
        f"cc={COUNTRY_CODE}&lang={LANGUAGE_CODE}&q={query}&size=1&type=PRODUCT"
    )
    headers = {"Authorization": f"Bearer {token}"}

    print(f"\nSearching for '{query}'...")

    try:
        response = requests.get(search_url, headers=headers)
        # Raise an exception for bad status codes (4xx or 5xx)
        response.raise_for_status()

        data = response.json()

        products = data.get("searchResultPage", {}).get("products", [])
        if not products:
            print("❌ No products found for that query.")
            return None

        return products[0] # Return the first product in the list

    except requests.exceptions.RequestException as e:
        print(f"🚨 An error occurred during the search request: {e}")
        return None
    except json.JSONDecodeError:
        print("🚨 Failed to decode JSON from search response. The API might be down or changed.")
        return None

def check_inventory(item_no, store_id, token):
    """
    Checks the inventory for a specific item number at a specific store.
    Returns the availability information or None on failure.
    """
    inventory_url = (
        f"https://api.ingka.ikea.com/cia/availabilities/v1/{COUNTRY_CODE}/stores/{store_id}/items"
        f"?itemNos={item_no}"
    )
    headers = {"Authorization": f"Bearer {token}"}

    print(f"Checking stock for item {item_no} at store {store_id}...")

    try:
        response = requests.get(inventory_url, headers=headers)
        response.raise_for_status()

        data = response.json()

        availabilities = data.get("availabilities", [])
        if not availabilities:
            print("❌ Could not retrieve availability for this item.")
            return None

        return availabilities[0] # Return the availability for the requested item

    except requests.exceptions.RequestException as e:
        print(f"🚨 An error occurred during the inventory request: {e}")
        return None
    except json.JSONDecodeError:
        print("🚨 Failed to decode JSON from inventory response. The API might be down or changed.")
        return None

if __name__ == "__main__":
    # 0. Get the guest token first
    auth_token = get_guest_token()
    if not auth_token:
        print("Could not retrieve authentication token. Exiting.")
        exit()

    # 1. Get user input for the product search
    product_query = input("\nEnter the IKEA product you want to search for (e.g., 'billy bookcase'): ")

    # 2. Get user input for the store
    print("\nPlease choose a store to check inventory:")
    for key, store_info in STORES.items():
        print(f"  {key}: {store_info['name']}")

    store_choice = input("Enter the number of your chosen store: ")

    if store_choice not in STORES:
        print("Invalid store selection. Exiting.")
        exit()

    selected_store = STORES[store_choice]

    # 3. Perform the product search
    product_data = search_product(product_query, auth_token)

    if product_data:
        item_number = product_data.get("itemNo")
        product_name = product_data.get("name")
        product_type = product_data.get("typeName")
        price = product_data.get("price", {}).get("numeral", 0)
        currency = product_data.get("price", {}).get("currencyCode", "N/A")

        print(f"✅ Found Product: {product_name} ({product_type})")
        print(f"   - Item Number: {item_number}")
        print(f"   - Price: {price} {currency}")

        # 4. Perform the inventory check
        inventory_data = check_inventory(item_number, selected_store["id"], auth_token)

        if inventory_data:
            is_in_stock = inventory_data.get("isInStock", False)
            quantity = inventory_data.get("stock", {}).get("quantity", 0)

            print("\n--- Stock Information ---")
            print(f"Store: {selected_store['name']}")
            if is_in_stock:
                print(f"✅ In Stock")
                print(f"   - Available Quantity: {quantity}")
            else:
                print(f"❌ Out of Stock")
                restock_date = inventory_data.get("restockDate")
                if restock_date:
                    print(f"   - Estimated restock date: {restock_date}")
            print("-------------------------")



Requesting guest access token...
🚨 An error occurred while getting the token: 401 Client Error: Unauthorized for url: https://api.ingka.ikea.com/guest/token
Could not retrieve authentication token. Exiting.

Please choose a store to check inventory:
  1: Wembley
  2: Greenwich
  3: Croydon
  4: Manchester
  5: Edinburgh

Searching for 'bed'...
🚨 An error occurred during the search request: 401 Client Error: Unauthorized for url: https://api.ingka.ikea.com/cia/availabilities/search/v1?cc=gb&lang=en&q=bed&size=1&type=PRODUCT


In [1]:
import torch
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"CUDA version: {torch.version.cuda}")
print(f"Device count: {torch.cuda.device_count()}")

CUDA available: True
CUDA version: 12.1
Device count: 1


usage: ipykernel_launcher.py [-h] [-e ENV_NAME] [-t AE_MODEL_TYPE]
                             [-v AE_MODEL_VERSION] [-tet TRANS_MODEL_TYPE]
                             [-tev TRANS_MODEL_VERSION] [-d DEVICE]
                             [-ne EPOCHS] [-tne TRANS_EPOCHS] [-b BATCH_SIZE]
                             [-l LOG_FREQ] [-c CHECKPOINT_FREQ]
                             [-lr LEARNING_RATE] [-tlr TRANS_LEARNING_RATE]
                             [-m MAX_TRANSITIONS] [-rlc RECON_LOSS_CLIP] [-u]
                             [--unique_data] [-nl] [-nc] [-p] [-w] [-cml]
                             [-i EXTRA_INFO] [-pl N_PRELOAD] [-um] [-s]
                             [-ntu N_TRAIN_UNROLL] [-pld]
                             [--env_max_steps ENV_MAX_STEPS]
                             [--rl_unroll_steps RL_UNROLL_STEPS]
                             [--rl_train_steps RL_TRAIN_STEPS] [--exact_comp]
                             [--log_state_reprs] [--tags [TAGS ...]]
                 

SystemExit: 2

c:\users\sam\appdata\local\programs\python\python39\lib\site-packages\IPython\core\interactiveshell.py:3558: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
